In [1]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

onnxModelPath = '/datadrive/tensorrt/onnx_models/mobilenetv2-7.onnx'
tensorrt_file_name = '/datadrive/tensorrt/onnx_report/mobilenet.trt'

# create builder <- optimization profile, Takes a network in TensorRT and generates an engine that is optimized for the target platform. 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

In [2]:
trt_runtime

In [3]:
# trt(engine) model file version = 8.2.4 / tensorrt version = 8.4.1 <-  이렇게 버전 안 맞을때는 안됨
# 버전 8.4.1 로 trtexec 다시 해서 모델 만들어서 하면 됨!

with open(tensorrt_file_name, 'rb') as f:
    engine_data = f.read()
engine = trt_runtime.deserialize_cuda_engine(engine_data)

[07/01/2022-07:37:48] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0


In [4]:
for binding in engine:
    print(binding,engine.get_binding_shape(binding))

input (1, 3, 224, 224)
output (1, 1000)


In [5]:
engine.num_bindings

2

In [6]:
engine.max_batch_size

[07/01/2022-07:37:52] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.


/tmp/ipykernel_42726/1690288493.py:1: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  engine.max_batch_size


1

In [7]:
engine.num_layers

55

In [8]:
engine.device_memory_size

7024640

In [ ]:
engine.refittable

In [ ]:
engine.name

In [ ]:
engine.profiling_verbosity

In [9]:
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()

In [201]:
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()

for binding in engine:
    print(engine.get_binding_shape(binding))
    print(engine.max_batch_size)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    print(size)
    dtype = trt.nptype(engine.get_binding_dtype(binding)) #float 32 안됨
    print(dtype)
    host_mem = cuda.pagelocked_empty(size, dtype)
    print(len(host_mem))
    print(type(host_mem))
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    print(int(device_mem))
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))

(1, 3, 416, 416)
1
519168
<class 'numpy.float32'>
519168
<class 'numpy.ndarray'>
140176854614016
(1, 125, 13, 13)
1
21125
<class 'numpy.float32'>
21125
<class 'numpy.ndarray'>
140176848407552
[07/07/2022-08:20:25] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/07/2022-08:20:25] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/07/2022-08:20:25] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/07/2022-08:20:25] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinition

/tmp/ipykernel_42726/3418207830.py:8: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  print(engine.max_batch_size)
/tmp/ipykernel_42726/3418207830.py:9: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size


In [11]:
inputs

[Host:
 [0. 0. 0. ... 0. 0. 0.]
 Device:

In [12]:
outputs

[Host:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [13]:
bindings

[140186308922368, 140186308575232]

In [14]:
stream

In [15]:
context = engine.create_execution_context()

[07/01/2022-07:38:14] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0


In [16]:
context

In [37]:
import torch
import numpy as np
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


input_data = torch.randn(1,3, 224, 224).to(device)
input_data = to_numpy(input_data)

In [38]:
input_data

array([[[[ 0.08432246, -0.1058276 , -0.62954396, ...,  0.17907499,
          -1.1518197 ,  0.5685717 ],
         [-0.07185556,  0.64454466,  0.5704944 , ...,  0.17724264,
           1.0886189 , -0.55334747],
         [-0.5438037 , -0.75219506, -1.5515512 , ..., -0.88678896,
           0.12020984,  0.01209653],
         ...,
         [-0.3059578 ,  0.33755505,  2.3349192 , ...,  0.66597223,
           0.1955215 ,  1.4519386 ],
         [-1.1752409 ,  2.2817771 , -0.1215327 , ..., -0.5537824 ,
          -0.6690678 , -0.68045217],
         [-1.6394802 , -1.424426  ,  2.3718534 , ...,  1.7265902 ,
          -0.7573196 ,  1.3781017 ]],

        [[ 2.7504265 , -0.47550508, -1.035468  , ..., -0.6198801 ,
           2.159974  , -1.6373792 ],
         [ 2.1746905 , -1.6749792 , -0.42819417, ..., -0.16059761,
           0.9148622 ,  0.35861462],
         [ 0.37983966, -0.3215695 , -0.5373664 , ...,  0.08511668,
           1.1830088 ,  1.9882578 ],
         ...,
         [ 2.1545713 , -0.7136638 

In [1]:
def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

In [17]:
# This function is generalized for multiple inputs/outputs for full dimension networks.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

In [ ]:
hosts = [input.host for input in inputs]
trt_types = [trt.int32, trt.int32, trt.int32]

In [43]:
for input in inputs:
    print(input)
    print(type(input))
    print(input.device)
    print(len(input.host))
    print(type(input.host))
    np.copyto(input.host,np.ravel(input_data))

Host:
[0. 0. 0. ... 0. 0. 0.]
Device:
<class '__main__.HostDeviceMem'>
150528
<class 'numpy.ndarray'>


In [47]:
context

In [48]:
bindings

[140186308922368, 140186308575232]

In [44]:
trt_outputs = do_inference_v2(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)

[07/01/2022-07:44:34] [TRT] [E] 1: [convolutionRunner.cpp::execute::213] Error Code 1: Cask (Cask convolution execution)
[07/01/2022-07:44:34] [TRT] [E] 1: [checkMacros.cpp::catchCudaError::278] Error Code 1: Cuda Runtime (invalid resource handle)


In [13]:
# 싱글 thread 에서 처리해야함. https://forums.developer.nvidia.com/t/tensorrt-do-inference-error/77055/3

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit # https://forums.developer.nvidia.com/t/logicerror-explicit-context-dependent-failed-invalid-device-context-no-currently-active-context/64149
import torch
import numpy as np

#onnxModelPath = '/datadrive/tensorrt/test_onnx'
tensorrt_file_name = '/datadrive/tensorrt/tinyyolov2-8.trt'

# create builder <- optimization profile, Takes a network in TensorRT and generates an engine that is optimized for the target platform. 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

with open(tensorrt_file_name, 'rb') as f:
    engine_data = f.read()
engine = trt_runtime.deserialize_cuda_engine(engine_data)
print(engine)

class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()
context = engine.create_execution_context()

for binding in engine:
    print(engine.get_binding_shape(binding))
    print(engine.max_batch_size)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    print(size)
    dtype = trt.nptype(engine.get_binding_dtype(binding)) #float 32 안됨
    print(dtype)
    host_mem = cuda.pagelocked_empty(size, dtype)
    print(len(host_mem))
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    print(int(device_mem))
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))
       
# for inp in inputs:
#     print(inp.host.shape)

# for oup in outputs:
#     print(oup.host.shape)
#     print(oup.device)
#     print(stream)

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


input_data = torch.randn(1,3, 416, 416).to(device)
input_data = to_numpy(input_data)

for input in inputs:
    np.copyto(input.host,np.ravel(input_data))

# This function is generalized for multiple inputs/outputs for full dimension networks.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

trt_outputs = do_inference_v2(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)

(1, 3, 416, 416)
1
519168
<class 'numpy.float32'>
519168
140093977264128
(1, 125, 13, 13)
1
21125
<class 'numpy.float32'>
21125
140093997177344
[07/08/2022-08:27:57] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/08/2022-08:27:57] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/08/2022-08:27:57] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/08/2022-08:27:57] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This functio

/tmp/ipykernel_2093446/773277678.py:42: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  print(engine.max_batch_size)
/tmp/ipykernel_2093446/773277678.py:43: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size


In [185]:
output_shapes=[(1, 125, 13, 13)]
trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs,output_shapes)]

In [189]:
trt_outputs[0].shape

(1, 125, 13, 13)

In [187]:
trt_outputs[0].shape

(1, 125, 13, 13)

In [204]:
stream = cuda.Stream()
stream.ptr

AttributeError: 'Stream' object has no attribute 'ptr'

In [164]:
trt_outputs[0:][0] 

array([-1.56515121e+00,  8.79994214e-01,  1.44220531e+00,  6.32113218e-01,
        2.04498124e+00,  3.09517169e+00,  2.23589587e+00,  7.49750733e-02,
       -6.05003946e-02, -1.34477878e+00, -8.74073029e-01,  1.42090356e+00,
        1.12184203e+00,  7.01641440e-01,  2.76940525e-01,  1.66452599e+00,
        9.67540860e-01,  7.01651633e-01,  3.51729250e+00,  3.27162713e-01,
        1.59683645e-01,  1.11701861e-01,  7.36026615e-02,  5.57369590e-01,
       -4.28803980e-01, -1.12783074e+00,  3.00756395e-01,  7.66739845e-01,
       -5.59235036e-01, -1.02722609e+00, -1.25582433e+00,  1.08535779e+00,
       -2.44438195e+00, -3.58368576e-01,  8.28549206e-01, -2.27776170e+00,
       -7.86838889e-01, -2.33976260e-01,  1.55161291e-01, -7.64069259e-02,
        1.25982094e+00, -6.06426895e-01,  1.17203593e+00, -1.10183573e+00,
        5.14742911e-01, -5.04573882e-01,  2.04487610e+00, -2.26586461e+00,
        5.46271503e-02, -2.06705260e+00, -5.75847387e-01, -2.49466825e+00,
        4.59031105e-01, -

In [54]:
trt_outputs

[array([-1.83780634e+00,  1.11267281e+00,  1.33001983e+00,  6.84026659e-01,
         1.87992597e+00,  3.03544497e+00,  2.35607076e+00, -4.77835506e-01,
        -5.51684141e-01, -1.68703604e+00, -8.21623921e-01,  1.35264337e+00,
         1.49944818e+00,  5.92964292e-01,  5.38729012e-01,  1.67636716e+00,
         1.02014148e+00,  6.25939906e-01,  3.53781080e+00,  1.53281301e-01,
         5.26170552e-01, -1.02307782e-01, -3.66636842e-01,  1.98492572e-01,
        -7.22454786e-01, -1.34811616e+00,  2.23169073e-01,  8.66105974e-01,
        -7.85946488e-01, -1.21605396e+00, -9.86493647e-01,  1.28093338e+00,
        -2.17999625e+00, -4.76416647e-01,  1.35433987e-01, -2.03166342e+00,
        -6.85697317e-01, -6.57868236e-02, -1.06951468e-01, -3.93141627e-01,
         1.20132041e+00, -9.81076360e-01,  8.66410494e-01, -1.44313490e+00,
         2.39185169e-01, -9.36252654e-01,  2.06716061e+00, -2.71192098e+00,
        -4.70329762e-01, -2.00526094e+00, -3.69234920e-01, -2.60790348e+00,
         4.3

In [55]:
# 싱글 thread 에서 처리해야함. https://forums.developer.nvidia.com/t/tensorrt-do-inference-error/77055/3

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit # https://forums.developer.nvidia.com/t/logicerror-explicit-context-dependent-failed-invalid-device-context-no-currently-active-context/64149
import torch
import numpy as np

#onnxModelPath = '/datadrive/tensorrt/onnx_models/mobilenetv2-7.onnx'
tensorrt_file_name = '/datadrive/tensorrt/mobilenet.trt'

# create builder <- optimization profile, Takes a network in TensorRT and generates an engine that is optimized for the target platform. 
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

with open(tensorrt_file_name, 'rb') as f:
    engine_data = f.read()
engine = trt_runtime.deserialize_cuda_engine(engine_data)


class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()
context = engine.create_execution_context()

for binding in engine:
    print(engine.get_binding_shape(binding))
    print(engine.max_batch_size)
    size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
    print(size)
    dtype = trt.nptype(engine.get_binding_dtype(binding)) #float 32 안됨
    print(dtype)
    host_mem = cuda.pagelocked_empty(size, dtype)
    print(len(host_mem))
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    print(int(device_mem))
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))
       

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


input_data = torch.randn(1,3, 224, 224).to(device)
input_data = to_numpy(input_data)

for input in inputs:
    np.copyto(input.host,np.ravel(input_data))

# This function is generalized for multiple inputs/outputs for full dimension networks.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference_v2(context, bindings, inputs, outputs, stream):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

trt_outputs = do_inference_v2(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)

[07/04/2022-03:05:12] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
(1, 3, 224, 224)
[07/04/2022-03:05:12] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
1
150528
<class 'numpy.float32'>
150528
140177481814016
(1, 1000)
1
1000
<class 'numpy.float32'>
1000
140177483397120
[07/04/2022-03:05:12] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/04/2022-03:05:12] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/04/2022-03:05:12] [TRT] [W] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[07/04/2022-03:

/tmp/ipykernel_42726/4032073572.py:42: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  print(engine.max_batch_size)
/tmp/ipykernel_42726/4032073572.py:43: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size


In [59]:
len(trt_outputs[0])

1000

In [79]:
trt_outputs[0][np.newaxis,:].shape

(1, 1000)

In [122]:
import numpy as np

a1 = np.array([[1,2,3,4,5],[1,2,4,6,7]])
a2 = np.array([[1,2,3,4,5],[1,2.01,4,5,7]])
a1.shape
#a3 = np.array([1,3,4.001,5,7])
print(np.isclose(a1,a2,atol=0.01))
res = np.isclose(a1,a2,atol=0.01)
#print(np.isclose(a3,a2))

[[ True  True  True  True  True]
 [ True  True  True False  True]]


In [134]:
res.shape

(2, 5)

In [102]:
for r in res:
    print(r)

[ True  True  True False  True]


In [103]:
(res == True).sum()

4

In [104]:
(res == False).sum()

1

In [107]:
# 전체 원소의 개수
np.size(res)

5

In [108]:
#이렇게 반환값 주면 될듯
(res == True).sum() / np.size(res)

0.8

In [137]:
max_value = abs(a1-a2).max()

In [138]:
max_value

1.0

In [143]:
max_pos = np.where(abs(a1-a2) == max_value)

In [144]:
max_pos

(array([1]), array([3]))

In [145]:
max_pos[0][0],max_pos[1][0]

(1, 3)

In [148]:
(int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

0

In [131]:
abs(a1-a2).argmax()

8

In [136]:
a1.shape[1]

5

In [ ]:
(2,3)